# Potato disease identification (VGG-16)

## I.Prepocessing

#### 1. GPU/CPU Setting

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
from torchvision import transforms, datasets
import os,PIL,pathlib,warnings

warnings.filterwarnings("ignore")             

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

#### 2. Data Load

In [2]:
import os,PIL,random,pathlib

data_dir = './PotatoPlants/'
data_dir = pathlib.Path(data_dir)

data_paths  = list(data_dir.glob('*')) 
classeNames = [path.name for path in data_paths if path.is_dir()]
classeNames

['Late_blight', 'healthy', 'Early_blight']

In [3]:
train_transforms = transforms.Compose([
    transforms.Resize([224, 224]),  # Resize the input images to a uniform size
    # transforms.RandomHorizontalFlip(), # Randomly flip the images horizontally
    transforms.ToTensor(),          # Normalize to [0,1]
    transforms.Normalize(           # Standardization  
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # The mean and std are calculated by randomly sampling from the dataset.
])

test_transform = transforms.Compose([
    transforms.Resize([224, 224]),  # Resize the input images to a uniform size
    transforms.ToTensor(),          # Normalize to [0,1]
    transforms.Normalize(           # Standardization 
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])  # The mean and std are calculated by randomly sampling from the dataset.
])

total_data = datasets.ImageFolder("./PotatoPlants/", transform=train_transforms)
total_data

Dataset ImageFolder
    Number of datapoints: 2152
    Root location: ./PotatoPlants/
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [4]:
total_data.class_to_idx

{'Early_blight': 0, 'Late_blight': 1, 'healthy': 2}

#### 3. Data Split

In [5]:
train_size = int(0.8 * len(total_data))
test_size  = len(total_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(total_data, [train_size, test_size])
train_dataset, test_dataset

(<torch.utils.data.dataset.Subset at 0x7f8ff8ce0d60>,
 <torch.utils.data.dataset.Subset at 0x7f8ff8cc1cd0>)

In [6]:
batch_size = 32

train_dl = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=1)
test_dl = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

In [7]:
for X, y in test_dl:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([32, 3, 224, 224])
Shape of y:  torch.Size([32]) torch.int64


## II. Manually Building the VGG-16 Mode

### VGG-16 Structure Explanation:
- 13 Convolutional Layers, denoted as `blockX_convX`
- 3 Fully Connected Layers, denoted as `fcX` and `predictions`
- 5 Pooling Layers, denoted as `blockX_pool`

![VGG-16 Architecture](https://neurohive.io/wp-content/uploads/2018/11/vgg16-neural-network.jpg)

#### 1. Build the model

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        # Convolutional Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        # Convolutional Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        # Convolutional Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        # Convolutional Block 4
        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
        # Convolutional Block 5
        self.block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        )
      
        # Fully connected layers for classification
        self.classifier = nn.Sequential(
            nn.Linear(in_features=512*7*7, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=3)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
  
model = VGG16().to(device)
model

Using cpu device


VGG16(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, cei

#### 2. Check model details

In [10]:
# Count the number of model parameters and other metrics
import torchsummary as summary
summary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## II. Train the model

#### 1. Write a training function

In [11]:
# Training loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # Size of the training dataset
    num_batches = len(dataloader)   # Number of batches (size/batch_size, rounded up)

    train_loss, train_acc = 0, 0  # Initialize training loss and accuracy
  
    for X, y in dataloader:  # Get images and their labels
        X, y = X.to(device), y.to(device)
      
        # Compute prediction and loss
        pred = model(X)          # Model output
        loss = loss_fn(pred, y)  # Calculate the difference between the model output and the true values (targets), the difference is the loss
      
        # Backpropagation
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()        # Backward pass
        optimizer.step()       # Update the model parameters
      
        # Record accuracy and loss
        train_acc  += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss += loss.item()
          
    train_acc  /= size
    train_loss /= num_batches

    return train_acc, train_loss

#### 2.Write the Test Function

The test function is similar to the training function; 
however, since gradient descent is not performed to update the network weights, there is no need to pass in the optimizer.

In [12]:
def test(dataloader, model, loss_fn):
    size        = len(dataloader.dataset)  # Size of the test dataset
    num_batches = len(dataloader)          # Number of batches (size/batch_size, rounded up)
    test_loss, test_acc = 0, 0

    # Disable gradient updates to save computation and memory during testing
    with torch.no_grad():
        for imgs, target in dataloader:
            imgs, target = imgs.to(device), target.to(device)

            # Compute loss
            target_pred = model(imgs)
            loss        = loss_fn(target_pred, target)

            test_loss += loss.item()
            test_acc  += (target_pred.argmax(1) == target).type(torch.float).sum().item()

    test_acc  /= size
    test_loss /= num_batches

    return test_acc, test_loss

### 3. Training

In [13]:
import copy

optimizer  = torch.optim.Adam(model.parameters(), lr=1e-4)  # Create the Adam optimizer
loss_fn    = nn.CrossEntropyLoss()  # Create the loss function

epochs     = 40

train_loss = []
train_acc  = []
test_loss  = []
test_acc   = []

best_acc = 0  # Initialize the best accuracy for model selection

for epoch in range(epochs):
  
    model.train()  # Set the model to training mode
    epoch_train_acc, epoch_train_loss = train(train_dl, model, loss_fn, optimizer)
  
    model.eval()  # Set the model to evaluation mode
    epoch_test_acc, epoch_test_loss = test(test_dl, model, loss_fn)
  
    # Save the best model to best_model
    if epoch_test_acc > best_acc:
        best_acc   = epoch_test_acc
        best_model = copy.deepcopy(model)
  
    train_acc.append(epoch_train_acc)
    train_loss.append(epoch_train_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
  
    # Get the current learning rate
    lr = optimizer.state_dict()['param_groups'][0]['lr']
  
    template = ('Epoch:{:2d}, Train_acc:{:.1f}%, Train_loss:{:.3f}, Test_acc:{:.1f}%, Test_loss:{:.3f}, Lr:{:.2E}')
    print(template.format(epoch+1, epoch_train_acc*100, epoch_train_loss, 
                          epoch_test_acc*100, epoch_test_loss, lr))
  
# Save the best model to a file
PATH = './best_model.pth'  # Filename for the saved parameters
torch.save(model.state_dict(), PATH)

print('Done')

KeyboardInterrupt: 

## III. Result Visualization

#### 1. Loss and Accuracy Graphs

In [ ]:
import matplotlib.pyplot as plt
# Hide warnings
import warnings
warnings.filterwarnings("ignore")               # Ignore warning messages
plt.rcParams['font.sans-serif']    = ['SimHei'] # To display Chinese labels correctly
plt.rcParams['axes.unicode_minus'] = False      # To display negative signs correctly
plt.rcParams['figure.dpi']         = 100        # Set resolution

epochs_range = range(epochs)

plt.figure(figsize=(12, 3))
plt.subplot(1, 2, 1)

plt.plot(epochs_range, train_acc, label='Training Accuracy')
plt.plot(epochs_range, test_acc, label='Test Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, test_loss, label='Test Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

#### 2. Predict on Specific Images

In [ ]:
from PIL import Image 

classes = list(total_data.class_to_idx)

def predict_one_image(image_path, model, transform, classes):
  

    test_img = Image.open(image_path).convert('RGB')
    plt.imshow(test_img)   

    test_img = transform(test_img)
    img = test_img.to(device).unsqueeze(0)
  

    model.eval()
    output = model(img)

    _,pred = torch.max(output,1)
    pred_class = classes[pred]
    print(f'predicted result：{pred_class}')

#### 3. Model Evaluation

In [ ]:
best_model.eval()
epoch_test_acc, epoch_test_loss = test(test_dl, best_model, loss_fn)

In [ ]:
epoch_test_acc, epoch_test_loss